In [20]:
import requests
from bs4 import BeautifulSoup
url = "http://www.theguardian.com/discussion/p/4fqc7"
r = requests.get(url)
html = r.text

soup = BeautifulSoup(html, "html.parser")
comments = soup.select(".d-comment__main")
comment_authors = soup.select(".d-comment__author")
print len (comments), " comments found in first page."
print len (comment_authors), " authors found in first page."

50  comments found in first page.
50  authors found in first page.


# Iterate the comments

In [24]:
parsed_comments = []
for comment, author in zip(comments, comment_authors):
    t = comment.select(".d-comment__body p")[0].text
    a = author['title']
    parsed_comments.append({"text": t, "author": a})
    
print parsed_comments[:6]

[{'text': u"I have lost count of the number of young people I have heard say they don't save, don't have a pension etc because the welfare system will only penalise for it, similarly elderly people who consider themselves fools for having savings or their own home.", 'author': u'DrJazz'}, {'text': u'This taxes the ill or needy. Surely it is better to just increase inheritance tax so that no-one has top pay in their lifetime but (especially unearned) wealth is taxed once someone has died?', 'author': u'darvit'}, {'text': u'he UK has committed to spending \xa31.162 trillion ', 'author': u'gtegte'}, {'text': u"The British say they love our NHS. But if you ask them to pay more taxes to fund it (especially for the care of the elderly) they don't want to know. How do they expect the NHS to carry on and why don't they think ahead to when they are old?", 'author': u'chelseachic'}, {'text': u"There is some merit in your argument. Although the post-war 'baby boomer' generation has been the most 